FINE TUNINING USING UNSLOTH

In [1]:
import shutil, os, subprocess
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
os.chdir('/content/drive/MyDrive/Colab_Notebooks/LLM')

In [3]:
!python --version

Python 3.10.12


In [4]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 24.3 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [5]:
import pandas as pd
from pprint import pprint

list_ds =[
{"instruction":"How you define sky.", "input":"the region of the atmosphere and outer space seen from the earth.", "output":"sky is region of atmosphere seen from earth"},
{"instruction":"What sport is most played.", "input":"a form of football played by two teams of eleven players with a round ball which may not be handled during play except by the goalkeepers.", "output":"Football is the most played game in world with 22 players on pitch"},
{"instruction":"Which Mountain is highest.", "input":"Mount Everest is Earth's highest mountain above sea level, located in the Mahalangur Himal sub-range of the Himalayas.", "output":"Mount Everest is the Highest Mountain in the world"},

{"instruction":"Which programming language is most common.", "input":"Javasctipt is an object-oriented computer programming language commonly used to create interactive effects within web browsers.", "output":"Javascript is most used programming language"},
{"instruction":"Which Language is most common in world.", "input":"English is the language of England, widely used in many varieties throughout the world.", "output":"English is the most common language in the World"},

 {"instruction":"Which company is the Acquirer.", "input":"New York, NY - Empire Innovations Inc, a leading technology conglomerate, announced today its plans to acquire Unique Ventures LLC, a prominent venture capital firm specializing in disruptive startups.", "output":"Empire Innovations Inc"},
{"instruction":"When Dinosaurs extinct.", "input":"66 million years ago, over a relatively short time, dinosaurs disappeared completely (except for birds).", "output":"66 million years ago dinosaurs dissapeared."}
]

pprint(list_ds)

[{'input': 'the region of the atmosphere and outer space seen from the earth.',
  'instruction': 'How you define sky.',
  'output': 'sky is region of atmosphere seen from earth'},
 {'input': 'a form of football played by two teams of eleven players with a '
           'round ball which may not be handled during play except by the '
           'goalkeepers.',
  'instruction': 'What sport is most played.',
  'output': 'Football is the most played game in world with 22 players on '
            'pitch'},
 {'input': "Mount Everest is Earth's highest mountain above sea level, located "
           'in the Mahalangur Himal sub-range of the Himalayas.',
  'instruction': 'Which Mountain is highest.',
  'output': 'Mount Everest is the Highest Mountain in the world'},
 {'input': 'Javasctipt is an object-oriented computer programming language '
           'commonly used to create interactive effects within web browsers.',
  'instruction': 'Which programming language is most common.',
  'output': 'J

In [6]:
%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()
# Must install separately since Colab has torch 2.2.1, which breaks packages
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass

In [7]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 1024 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.


In [8]:

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj","gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)



config.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.4
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/131 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Unsloth 2024.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [9]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

import datasets
import pandas as pd
# Create a dataset from the array of dictionaries

df = pd.DataFrame(list_ds)
dataset = datasets.Dataset.from_pandas(df)

# Print the dataset
print(dataset)
dataset = dataset.map(formatting_prompts_func, batched = True,)
print(dataset)

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 7
})


Map:   0%|          | 0/7 [00:00<?, ? examples/s]

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 7
})


In [10]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 50,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=2):   0%|          | 0/7 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [11]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
5.605 GB of memory reserved.


In [12]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 7 | Num Epochs = 50
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 50
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,3.000700
2,2.982100
3,2.915900
4,2.674300
5,2.241900
6,1.939800
7,1.579600
8,1.234800
9,0.954400
10,0.781200


In [13]:
model.save_pretrained_merged("model_mna", tokenizer, save_method = "merged_16bit",)

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which will take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 5.7G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 7.46 out of 12.67 RAM for saving.


 50%|█████     | 16/32 [01:00<01:00,  3.81s/it]


RuntimeError: File _unsloth_temporary_saved_buffers/model.layers.16.mlp.up_proj.weight.pt cannot be opened.

In [14]:
mna_news_input = """Dinosaurs extinct 66 millions years ago on earth but birds are survived from that extinction and lived until today."""

mna_news_instruction = """What is the extinction date. """
mna_news_instruction = """which animals survived. """

In [15]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass


In [16]:
alpaca_prompt.format(mna_news_instruction, mna_news_input, "")

'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nwhich animals survived. \n\n### Input:\nDinosaurs extinct 66 millions years ago on earth but birds are survived from that extinction and lived until today.\n\n### Response:\n'

In [17]:
inputs = tokenizer([alpaca_prompt.format(mna_news_instruction, mna_news_input, "")], return_tensors = "pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens = 90)
tokenizer.batch_decode(outputs)[0]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nwhich animals survived. \n\n### Input:\nDinosaurs extinct 66 millions years ago on earth but birds are survived from that extinction and lived until today.\n\n### Response:\nBirds survived the dinosaur extinction.<|end_of_text|>'